# Analysis

## Logical Analysis

In [ ]:
import math

capital_letters = 26
small_letters = 26
digits = 10
special_characters = 33
all_characters = 95
l = 13
passphrase = (math.comb(l,4)) * capital_letters * small_letters * digits * special_characters * all_characters ** (l-4)

mnemonic_BIP39 = 2048
unique_biometric = 8000000000

possible_combinations = passphrase * mnemonic_BIP39 * unique_biometric
print(possible_combinations)
print(2**128)

1647020726677939641600000000000000000000
340282366920938463463374607431768211456


## Implementation Analysis

In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00


In [2]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=863e65c043f9bfb16c932b1ed39cc24cb5bae10d1d60f0b663b85dfd2f50ddac
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
import os
import sys
from skimage.morphology import skeletonize, thin
import cv2
import numpy
from skimage.morphology import skeletonize
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import hashlib
from deepface import DeepFace

24-08-21 21:00:34 - Directory /root/.deepface has been created
24-08-21 21:00:34 - Directory /root/.deepface/weights has been created


### Fingerprint Analysis

In [ ]:
import os
import cv2
import numpy
import hashlib
from skimage.morphology import skeletonize

def get_descriptors(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    img = numpy.array(img, dtype=numpy.uint8)
    # Threshold
    ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    # Normalize to 0 and 1 range
    img[img == 255] = 1

    # Thinning
    skeleton = skeletonize(img)
    skeleton = numpy.array(skeleton, dtype=numpy.uint8)
    # Harris corners
    harris_corners = cv2.cornerHarris(img, 3, 3, 0.04)
    harris_normalized = cv2.normalize(harris_corners, 0, 255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32FC1)
    threshold_harris = 125
    # Extract keypoints
    keypoints = []
    for x in range(0, harris_normalized.shape[0]):
        for y in range(0, harris_normalized.shape[1]):
            if harris_normalized[x][y] > threshold_harris:
                keypoints.append(cv2.KeyPoint(y, x, 1))
    # Define descriptor
    orb = cv2.ORB_create()
    # Compute descriptors
    _, des = orb.compute(img, keypoints)

    # Convert descriptors to a string
    if des is not None:
        descriptors_str = ' '.join(map(str, des.flatten()))
    else:
        descriptors_str = ''

    return descriptors_str

def process_fingerprint_directory(directory, output_file):
    with open(output_file, 'w') as f:
        for filename in os.listdir(directory):
            if filename.endswith('.BMP'):
                img_path = os.path.join(directory, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                keypoints_str = get_descriptors(img)

                # Compute SHA-256 hash of the concatenated keypoints string
                sha256_hash = hashlib.sha256(keypoints_str.encode()).hexdigest()

                # Write the result to the output file
                f.write(f"{filename} {sha256_hash}\n")
                print(f"Processed {filename}")

fingerprint_directory = "/content/fingerprint"
for i in range(10):
  output_file = "/content/fingerprint" + str(i) + ".txt"
  process_fingerprint_directory(fingerprint_directory, output_file)
  print(f'round: {i}')


Streaming output truncated to the last 5000 lines.
Processed 45__M_Left_ring_finger.BMP
Processed 582__M_Left_index_finger.BMP
Processed 24__F_Left_middle_finger.BMP
Processed 44__M_Left_thumb_finger.BMP
Processed 22__M_Right_ring_finger.BMP
Processed 566__M_Right_little_finger.BMP
Processed 32__M_Left_little_finger.BMP
Processed 586__M_Right_ring_finger.BMP
Processed 40__F_Right_ring_finger.BMP
Processed 570__M_Right_little_finger.BMP
Processed 543__M_Right_thumb_finger.BMP
Processed 538__M_Right_middle_finger.BMP
Processed 26__M_Left_little_finger.BMP
Processed 26__M_Right_little_finger.BMP
Processed 5__M_Right_index_finger.BMP
Processed 546__M_Left_ring_finger.BMP
Processed 581__F_Right_thumb_finger.BMP
Processed 45__M_Left_index_finger.BMP
Processed 535__M_Left_middle_finger.BMP
Processed 583__M_Left_index_finger.BMP
Processed 17__M_Left_thumb_finger.BMP
Processed 29__F_Left_thumb_finger.BMP
Processed 2__F_Right_index_finger.BMP
Processed 18__M_Right_ring_finger.BMP
Processed 29__F

In [ ]:
import os
import hashlib

def hash_file(file_path):
    """Calculate the SHA-256 hash of the file content."""
    sha256_hash = hashlib.sha256()
    with open(file_path, 'rb') as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

def check_files_identical(directory):
    """Check if all files fingerprint0.txt to fingerprint9.txt have the same content."""
    base_filename = "fingerprint"
    file_hashes = []

    for i in range(10):
        file_name = f"{base_filename}{i}.txt"
        file_path = os.path.join(directory, file_name)

        if not os.path.exists(file_path):
            print(f"{file_name} does not exist.")
            return False

        file_hash = hash_file(file_path)
        file_hashes.append(file_hash)

    # Check if all hashes are identical
    if all(h == file_hashes[0] for h in file_hashes):
        print("All files have the same content.")
        return True
    else:
        print("Files do not have the same content.")
        return False

directory = "/content"  # Directory containing the fingerprint[i].txt files
check_files_identical(directory)

All files have the same content.


True